In [18]:
import time
import pdb

import requests, json 
import flask

import gmaps
with open('apikey.txt') as f:
    api_key = f.readline().strip()
    f.close
    
gmaps.configure(api_key=api_key)



In [20]:
flask.__version__

'1.1.1'

In [2]:
new_york_coordinates = (40.75, -74.00)
gmaps.figure(center=new_york_coordinates, zoom_level=12)


Figure(layout=FigureLayout(height='420px'))

In [3]:
ratings = {}
addresses = {}
URL = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
# https://developers.google.com/places/web-service/search#TextSearchRequests
MIN_USER_RATINGS = 10
MIN_RATING = 4

# home
location = '40.6875627,-74.0035107'
# grand army plaza
# location = '40.671872,-73.972544'
# union st and 4th ave
# location ='40.677485,-73.983310'
# bay ridge
# location = "40.624468,-74.0487134"

#rankby='prominence'
rankby='distance'
keyword='coffee'
ltype='establishment'
radius='3000'
#https://developers.google.com/places/web-service/supported_types

In [14]:
def get_gmaps_results(api_key, location, **kwargs):

    request_url = URL + '?key=' + api_key
    request_url += '&location=' + location
    for name, val in kwargs.items():
        request_url += '&' + name + '=' + val
    r = requests.get(request_url)
    j = r.json()
    return j

def get_next_page(api_key, next_page_token):
    r = requests.get(URL + '?pagetoken=' + next_page_token +
                        '&key=' + api_key)
    for i in range(10):
        j = r.json()
        if not j['results']: # wait for next page to be available
            time.sleep(5)
            continue
        else:
            return j

def process_results(results):
    global ratings, addresses
    for i, res in enumerate(results): 
        if res['user_ratings_total'] < MIN_USER_RATINGS:
            continue
        if res['rating'] < MIN_RATING:
            continue
        addresses[res['name']] = res['vicinity']
        ratings[res['name']] = res['rating']
        print(res['name'], res['rating'])
    return i+1

In [15]:
def runquery(api_key, location, **kwargs):
    j = get_gmaps_results(api_key, location, **kwargs)
    results = j['results']
    totalresults = 0
    
    totalresults += process_results(results)
    
    while 'next_page_token' in j:
        next_page_token = j['next_page_token']
        time.sleep(5)
        j = get_next_page(api_key, next_page_token)
        results = j['results'] 
        totalresults += process_results(results)
        
    return totalresults

In [16]:
# use either rankby or radius
# j = get_gmaps_results(api_key, location, keyword=keyword, ltype=ltype, radius=radius)
runquery(api_key, location, keyword=keyword, ltype=ltype, rankby=rankby)


Fare and Folk 4.5
Espresso Me 4.4
Kaigo Coffee Room 4.8
Henry's Local 4.2
Swallow Cafe 4.3
Marquet Patisserie 4.2
D'Amico Coffee 4.6
Starbucks 4.1
East One Coffee Roasters 4.5
Stumptown Coffee Roasters 4.6
Velvette Brew 4.7
About coffee 4.8
Starbucks 4.1
Starbucks 4.1
Mia's 4.6
Blue Bottle Coffee 4.5
Maison Kayser 4.3
Brooklyn Habit 4.6
Gregorys Coffee 4.4
Seeds of love 4
Bien Cuit 4.3
Konditori 4.2
Nerd be cool. Espresso Lab 4.6
Absolute Coffee 4.4
Starbucks 4
The Little Sweet Café 4.5
Baked 4.5
Joe Coffee Company 4.3
Commons Cafe 4.5
Casella Bagel Coffee Shop 4.3
Devoción 4.4
dot & line 4.7
Abbotsford Road Coffee Specialists 4.7
Ground Central Coffee Company 4.4
Stone Street Coffee Company Cafe 4
Bluestone Lane DUMBO Café 4.3
Cafe Grumpy - Financial District 4.4
Gregorys Coffee 4.1
Hungry Ghost Coffee 4.8
The Brooklyn Cat Cafe 4.6
Regular Visitors 4.5
Le Petit Cafe 4.3
Building On Bond 4.2
Allegro Coffee Roasters 4.2
Bluestone Lane Water Street Coffee Shop 4.2
Starbucks 4
Gregorys Co

60

In [9]:
jarray = []
for name, rating in reversed(sorted(ratings.items(), key=lambda item: item[1])):
    print (rating, name, addresses[name])
    jarray.append([name, addresses[name], rating])
    


4.8 Coffee Project NY 78 Rockwell Pl, Brooklyn
4.8 Hungry Ghost Coffee 80 Dekalb Ave, Brooklyn
4.8 About coffee 187 State St, Brooklyn
4.8 Kaigo Coffee Room 139 Bridge Park Dr, Brooklyn
4.7 Abbotsford Road Coffee Specialists 573 Sackett St, Brooklyn
4.7 dot & line 253 Bergen St, Brooklyn
4.7 Velvette Brew 251 Smith St, Brooklyn
4.6 The Brooklyn Cat Cafe 76 Montague St, Brooklyn
4.6 Nerd be cool. Espresso Lab 518 Court St, Brooklyn
4.6 Brooklyn Habit 405 Smith St, Brooklyn
4.6 Mia's 139 Smith St, Brooklyn
4.6 Stumptown Coffee Roasters 212b Pacific St, Brooklyn
4.6 D'Amico Coffee 309 Court St, Brooklyn
4.5 Blue Spoon Coffee Company 90 William St, New York
4.5 Black Fox Coffee 70 Pine St, New York
4.5 Brooklyn Roasting Company 25 Jay St, Brooklyn
4.5 La Colombe Coffee Roasters 67 Wall St, New York
4.5 WHITE NOISE COFFEE CO. 57 Smith St, Brooklyn
4.5 Regular Visitors 149 Smith St, Brooklyn
4.5 Ninth Street Espresso 333 Douglass St #3115, Brooklyn
4.5 Commons Cafe 388 Atlantic Ave, Brooklyn

In [ ]:
import flask
app = flask.Flask(__name__)
app.config["DEBUG"] = True

@app.route('/', methods=['GET'])
def home():
    return "<h1>Hello</p>"

app.run(port=8502)

In [ ]:
flask.jsonify(jarray)

In [ ]:
def testfunc(**kwargs):
    for name, val in kwargs.items():
        print (name, val)

In [ ]:
testfunc(manza='tetas', ola='raman')